In [1]:
import json

# Path to the original train.json file
train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/train.json'

# Load the train.json file
with open(train_json_path, 'r') as file:
    train_data = json.load(file)

# Display the first few entries to verify the content
print(train_data[:3])  # Adjust the number of entries to display based on your need


[{'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 0}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 1}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 2}]


In [2]:
import os

def load_detection_json(img_folder):
    # Path to the folder containing detection.json
    detection_json_path = os.path.join('/Users/jeffreysherer/Dissertation/data/imgs-1', img_folder, 'detection.json')
    
    # Load the detection.json file
    if os.path.exists(detection_json_path):
        with open(detection_json_path, 'r') as file:
            detection_data = json.load(file)
        return detection_data
    else:
        return None  # Return None if file does not exist

# Test the function with an example folder
example_folder = 'xmlab1'
example_detection = load_detection_json(example_folder)
print(example_detection)  # Display the content of the detection.json for verification


[{'Liver': [54.0, 106.0, 30.0, 31.0]}]


In [3]:
# Function to add detection data to train_data entries
def integrate_detection_data(train_data):
    for item in train_data:
        # Extract the folder name from img_name
        folder_name = item['img_name'].split('/')[0]
        
        # Load detection data for the corresponding folder
        detection_data = load_detection_json(folder_name)
        
        # If detection data exists, add it to the train data item
        if detection_data:
            item['detection'] = detection_data
    
    return train_data

# Update train_data with detection information
updated_train_data = integrate_detection_data(train_data)

# Print a few updated entries to verify that detection data has been added correctly
print(updated_train_data[:3])  # Adjust the number of entries to display based on your need


[{'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 0, 'detection': [{'Liver': [54.0, 106.0, 30.0, 31.0]}]}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 1, 'detection': [{'Liver': [54.0, 106.0, 30.0, 31.0]}]}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 2, 'detectio

In [4]:
import os
import json
import uuid  # To generate unique IDs

# Base path where the original train.json is located
base_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0'

# Directory containing the image folders with detection.json files
image_directory = '/Users/jeffreysherer/Dissertation/data/imgs-1'

# Path to the original train.json file
train_json_path = os.path.join(base_path, 'train.json')

# Load the train.json file
with open(train_json_path, 'r') as file:
    train_data = json.load(file)

# Function to update the data with bounding box coordinates
def update_data_with_bounding_boxes(data):
    for entry in data:
        img_folder = os.path.dirname(entry['img_name'])
        detection_json_path = os.path.join(image_directory, img_folder, 'detection.json')

        if os.path.exists(detection_json_path):
            with open(detection_json_path, 'r') as file:
                detection_data = json.load(file)
            entry['detection'] = detection_data

    return data

# Update the train data with bounding box coordinates
updated_train_data = update_data_with_bounding_boxes(train_data)

# Function to transform the data to the required format
def transform_data(data):
    transformed_data = []
    for entry in data:
        unique_id = str(uuid.uuid4())  # Generate a unique ID for each entry
        img_path = entry['img_name']  # Use the img_name as the image path
        question = entry['question']
        answer = entry['answer']
        
        transformed_entry = {
            "id": unique_id,
            "image": img_path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": answer
                }
            ]
        }
        if 'detection' in entry:
            transformed_entry['detection'] = entry['detection']
        transformed_data.append(transformed_entry)
    return transformed_data

# Transform the train data
transformed_train_data = transform_data(updated_train_data)

# Path to save the final train.json file
final_train_json_path = os.path.join(augmented_dir, 'final_train.json')

# Write the transformed train data to a new JSON file
with open(final_train_json_path, 'w') as file:
    json.dump(transformed_train_data, file, indent=4)

print("Final train.json has been saved to", final_train_json_path)


Updated train.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_train.json


In [5]:
import json
import uuid  # To generate unique IDs

# Assuming 'updated_train_data' is already defined from the previous cell

# Transform the data to the required format
transformed_data = []
for entry in updated_train_data:
    unique_id = str(uuid.uuid4())  # Generate a unique ID for each entry
    img_path = entry['img_name']  # Use the img_name as the image path
    question = entry['question']
    answer = entry['answer']
    
    transformed_entry = {
        "id": unique_id,
        "image": img_path,
        "conversations": [
            {
                "from": "human",
                "value": f"<image>\n{question}"
            },
            {
                "from": "gpt",
                "value": answer
            }
        ]
    }
    if 'detection' in entry:
        transformed_entry['detection'] = entry['detection']
    transformed_data.append(transformed_entry)

# Path to save the final transformed JSON file
final_train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/final_train.json'

# Write the transformed data to a new JSON file
with open(final_train_json_path, 'w') as file:
    json.dump(transformed_data, file, indent=4)

print("Final train.json has been saved to", final_train_json_path)


Transformed train.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_train.json


In [6]:
import json
import random

# Path to the transformed train.json file
transformed_train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_train.json'

# Load the transformed train.json file
with open(transformed_train_json_path, 'r') as file:
    transformed_data = json.load(file)

# Randomly sample 5 entries from the dataset to inspect
sampled_entries = random.sample(transformed_data, 5)

# Print the sampled entries
for entry in sampled_entries:
    print(json.dumps(entry, indent=4))


{
    "id": "a6ff7415-de7f-4270-af38-d750f726e01c",
    "image": "xmlab423/source.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\n\u989e\u53f6\u51fa\u73b0\u5728\u8fd9\u5f20\u56fe\u4e0a\u4e86\u5417?"
        },
        {
            "from": "gpt",
            "value": "\u6ca1\u6709"
        }
    ]
}
{
    "id": "533450c3-7d36-4dfc-a833-4c181d09e39b",
    "image": "xmlab275/source.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nWhere is the liver?"
        },
        {
            "from": "gpt",
            "value": "Left"
        }
    ]
}
{
    "id": "0dd1fc6e-d2f9-4516-8d18-7f2b9776cbd3",
    "image": "xmlab190/source.jpg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nDoes this image look abnormal?"
        },
        {
            "from": "gpt",
            "value": "Yes"
        }
    ]
}
{
    "id": "099138e8-ded5-4cac-9854-e98750e34

Now doing the trst adn validate json files


In [7]:
import json
import uuid  # To generate unique IDs
import os

# Paths to the original validate.json and test.json files
validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/validate.json'
test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/test.json'

# Load the validate.json file
with open(validate_json_path, 'r') as file:
    validate_data = json.load(file)

# Load the test.json file
with open(test_json_path, 'r') as file:
    test_data = json.load(file)

# Function to transform the data
def transform_data(data):
    transformed_data = []
    for entry in data:
        unique_id = str(uuid.uuid4())  # Generate a unique ID for each entry
        img_path = entry['img_name']  # Use the img_name as the image path
        question = entry['question']
        answer = entry['answer']
        
        transformed_entry = {
            "id": unique_id,
            "image": img_path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": answer
                }
            ]
        }
        transformed_data.append(transformed_entry)
    return transformed_data

# Transform validate and test data
transformed_validate_data = transform_data(validate_data)
transformed_test_data = transform_data(test_data)

# Display the first few entries to verify the content
print(json.dumps(transformed_validate_data[:3], indent=4))  # Adjust the number of entries to display based on your need
print(json.dumps(transformed_test_data[:3], indent=4))  # Adjust the number of entries to display based on your need

# Paths to save the transformed JSON files
transformed_validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_validate.json'
transformed_test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_test.json'

# Write the transformed validate data to a new JSON file
with open(transformed_validate_json_path, 'w') as file:
    json.dump(transformed_validate_data, file, indent=4)

# Write the transformed test data to a new JSON file
with open(transformed_test_json_path, 'w') as file:
    json.dump(transformed_test_data, file, indent=4)

print("Transformed validate.json has been saved to", transformed_validate_json_path)
print("Transformed test.json has been saved to", transformed_test_json_path)

# Directory containing the image folders with detection.json files
image_directory = '/Users/jeffreysherer/Dissertation/data/imgs-1'

# Function to update the data with bounding box coordinates
def update_data_with_bounding_boxes(transformed_data):
    for entry in transformed_data:
        # Extract the original image path
        img_path = entry['image']
        img_folder = os.path.dirname(img_path)
        detection_json_path = os.path.join(image_directory, img_folder, 'detection.json')
        
        if os.path.exists(detection_json_path):
            with open(detection_json_path, 'r') as file:
                detection_data = json.load(file)
            entry['detection'] = detection_data

    return transformed_data

# Update the transformed validate and test data with bounding box coordinates
updated_validate_data = update_data_with_bounding_boxes(transformed_validate_data)
updated_test_data = update_data_with_bounding_boxes(transformed_test_data)

# Paths to save the updated JSON files with bounding boxes
updated_validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_validate.json'
updated_test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_test.json'

# Write the updated validate data to a new JSON file
with open(updated_validate_json_path, 'w') as file:
    json.dump(updated_validate_data, file, indent=4)

# Write the updated test data to a new JSON file
with open(updated_test_json_path, 'w') as file:
    json.dump(updated_test_data, file, indent=4)

print("Updated transformed validate.json with bounding boxes has been saved to", updated_validate_json_path)
print("Updated transformed test.json with bounding boxes has been saved to", updated_test_json_path)


[
    {
        "id": "4c769b41-5e40-4b8c-b8ab-faed96555178",
        "image": "xmlab0/source.jpg",
        "conversations": [
            {
                "from": "human",
                "value": "<image>\nWhat modality is used to take this image?"
            },
            {
                "from": "gpt",
                "value": "MRI"
            }
        ]
    },
    {
        "id": "cdd7f275-d48f-459a-b27c-654dbcaa6dac",
        "image": "xmlab0/source.jpg",
        "conversations": [
            {
                "from": "human",
                "value": "<image>\nWhich part of the body does this image belong to?"
            },
            {
                "from": "gpt",
                "value": "Abdomen"
            }
        ]
    },
    {
        "id": "206f195c-88cf-41d7-a967-110232eeee17",
        "image": "xmlab0/source.jpg",
        "conversations": [
            {
                "from": "human",
                "value": "<image>\nWhat is the mr weighting in this im

In [8]:
import os
import json

# Directory containing the image folders with detection.json files
image_directory = '/Users/jeffreysherer/Dissertation/data/imgs-1'

# Function to update the data with bounding box coordinates
def update_data_with_bounding_boxes(transformed_data):
    for entry in transformed_data:
        # Extract the original image path
        img_path = entry['image']
        img_folder = os.path.dirname(img_path)
        detection_json_path = os.path.join(image_directory, img_folder, 'detection.json')
        
        if os.path.exists(detection_json_path):
            with open(detection_json_path, 'r') as file:
                detection_data = json.load(file)
            entry['detection'] = detection_data

    return transformed_data

# Load transformed validate and test data
transformed_validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_validate.json'
transformed_test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/transformed_test.json'

with open(transformed_validate_json_path, 'r') as file:
    transformed_validate_data = json.load(file)

with open(transformed_test_json_path, 'r') as file:
    transformed_test_data = json.load(file)

# Update the transformed validate and test data with bounding box coordinates
updated_validate_data = update_data_with_bounding_boxes(transformed_validate_data)
updated_test_data = update_data_with_bounding_boxes(transformed_test_data)

# Paths to save the updated JSON files with bounding boxes
updated_validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_validate.json'
updated_test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_test.json'

# Write the updated validate data to a new JSON file
with open(updated_validate_json_path, 'w') as file:
    json.dump(updated_validate_data, file, indent=4)

# Write the updated test data to a new JSON file
with open(updated_test_json_path, 'w') as file:
    json.dump(updated_test_data, file, indent=4)

print("Updated transformed validate.json with bounding boxes has been saved to", updated_validate_json_path)
print("Updated transformed test.json with bounding boxes has been saved to", updated_test_json_path)


Updated transformed validate.json with bounding boxes has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_validate.json
Updated transformed test.json with bounding boxes has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_transformed_test.json


In [9]:
# Paths to save the updated validate and test JSON files
augmented_folder = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented'
updated_validate_json_path = os.path.join(augmented_folder, 'BBF_validate.json')
updated_test_json_path = os.path.join(augmented_folder, 'BBF_test.json')

# Ensure the augmented folder exists
os.makedirs(augmented_folder, exist_ok=True)

# Write the updated validate data to a new JSON file
with open(updated_validate_json_path, 'w') as file:
    json.dump(updated_validate_data, file, indent=4)

# Write the updated test data to a new JSON file
with open(updated_test_json_path, 'w') as file:
    json.dump(updated_test_data, file, indent=4)

print("Updated validate.json has been saved to", updated_validate_json_path)
print("Updated test.json has been saved to", updated_test_json_path)


Updated validate.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_validate.json
Updated test.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_test.json
